In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [3]:
train = sqlContext.sql("SELECT * FROM train_csv")

In [4]:
test = sqlContext.sql("SELECT * FROM test_csv1")

In [5]:
from pyspark.sql.functions import isnan, when, count, col
train.select([count(when(col(c).isNull(), c)).alias(c) for c in train.columns]).show()

+---+-----+------+----+-----+
 id|title|author|text|label|
+---+-----+------+----+-----+
 0| 558| 0| 39| 0|
+---+-----+------+----+-----+

In [6]:
train = train.fillna(' ', subset=['title', 'text'])
train.select([count(when(col(c).isNull(), c)).alias(c) for c in train.columns]).show()

+---+-----+------+----+-----+
 id|title|author|text|label|
+---+-----+------+----+-----+
 0| 0| 0| 0| 0|
+---+-----+------+----+-----+

In [7]:
from pyspark.sql.functions import when
train = train.withColumn("author", \
              when(train["author"] == 'nan', ' ').otherwise(train["author"]))
train.show()

+---+--------------------+--------------------+--------------------+-----+
 id| title| author| text|label|
+---+--------------------+--------------------+--------------------+-----+
 0|House Dem Aide: W...| Darrell Lucus|House Dem Aide: W...| 1|
 1|FLYNN: Hillary Cl...| Daniel J. Flynn|Ever get the feel...| 0|
 2|Why the Truth Mig...| Consortiumnews.com|Why the Truth Mig...| 1|
 3|15 Civilians Kill...| Jessica Purkiss|Videos 15 Civilia...| 1|
 4|Iranian woman jai...| Howard Portnoy|Print 
An Iranian...| 1|
 5|Jackie Mason: Hol...| Daniel Nussbaum|In these trying t...| 0|
 6|Life: Life Of Lux...| |Ever wonder how B...| 1|
 7|Benoît Hamon Wins...| Alissa J. Rubin|PARIS — France...| 0|
 8|Excerpts From a D...| |Donald J. Trump i...| 0|
 9|A Back-Channel Pl...|Megan Twohey and ...|A week before Mic...| 0|
 10|Obama’s Organizin...| Aaron Klein|Organizing for Ac...| 0|
 11|BBC Comedy Sketch...| Chris Tomlinson|The BBC produced ...| 0|
 12|Russian Researche...| Amando Flavio|The mystery surro...| 1|
 13|US Officials See ...| Jason Ditz|Clinton Campaign ...| 1|
 14|Re: Yes, There Ar...| AnotherAnnie|Yes, There Are Pa...| 1|
 15|In Major League S...| Jack Williams|Guillermo Barros ...| 0|
 16|Wells Fargo Chief...|Michael Corkery a...|The scandal engul...| 0|
 17|Anonymous Donor P...| Starkman|A Caddo Nation tr...| 1|
 18|FBI Closes In On ...| The Doc|FBI Closes In On ...| 1|
 19|Chuck Todd: ’Buzz...| Jeff Poor|Wednesday after ...| 0|
+---+--------------------+--------------------+--------------------+-----+
only showing top 20 rows

In [8]:
 
from pyspark.sql.functions import isnan, when, count, col
test.select([count(when(col(c).isNull(), c)).alias(c) for c in test.columns]).show()

+---+-----+------+----+
 id|title|author|text|
+---+-----+------+----+
 0| 122| 0| 7|
+---+-----+------+----+

In [9]:
test = test.fillna(' ', subset=['title', 'text'])
test.select([count(when(col(c).isNull(), c)).alias(c) for c in test.columns]).show()

+---+-----+------+----+
 id|title|author|text|
+---+-----+------+----+
 0| 0| 0| 0|
+---+-----+------+----+

In [10]:
test = test.withColumn("author", \
              when(test["author"] == 'nan', ' ').otherwise(test["author"]))
test.show(5)

+-----+--------------------+--------------------+--------------------+
 id| title| author| text|
+-----+--------------------+--------------------+--------------------+
20800|Specter of Trump ...| David Streitfeld|PALO ALTO, Calif....|
20801|Russian warships ...| |Russian warships ...|
20802|#NoDAPL: Native A...| Common Dreams|Videos #NoDAPL: N...|
20803|Tim Tebow Will At...| Daniel Victor|If at first you d...|
20804|Keiser Report: Me...|Truth Broadcast N...|42 mins ago 1 Vie...|
+-----+--------------------+--------------------+--------------------+
only showing top 5 rows

In [11]:
df = train

In [12]:
from pyspark.sql.functions import concat_ws, concat
df = train
df = df.withColumn('combi_text', concat_ws(' ', df.title, df.author, df.text))
df = df.drop('title','author','text')
df.show(1,truncate=False)

+---+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
#Test data combining title, author and text
df_test = test
df_test = df_test.withColumn('combi_text', concat_ws(' ', df_test.title, df_test.author, df_test.text))
df_test = df_test.drop('title','author','text')
df_test.show(1,truncate=False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, RegexTokenizer,Word2Vec, HashingTF
from pyspark.ml.feature import StandardScaler
 
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
def nlpTransform(data):
  tokenizer = Tokenizer(inputCol="combi_text", outputCol="words")
  wordsData = tokenizer.transform(data)
  hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
  featurizedData = hashingTF.transform(wordsData)
  scaler = StandardScaler(inputCol="rawFeatures", outputCol="features", withStd=True, withMean=False)
  featureData = scaler.fit(featurizedData)
  featureD = featureData.transform(featurizedData)
  return featureD

In [15]:
data = nlpTransform(df)
data = data.drop('combi_text', 'words', 'rawFeatures')

In [16]:
data.show(3)

+---+-----+--------------------+
 id|label| features|
+---+-----+--------------------+
 0| 1|(262144,[1536,345...|
 1| 0|(262144,[336,881,...|
 2| 1|(262144,[1536,243...|
+---+-----+--------------------+
only showing top 3 rows

In [17]:
testData = nlpTransform(df_test)
testData = testData.drop('combi_text', 'words','rawFeatures')
testData.show(1)

+-----+--------------------+
 id| features|
+-----+--------------------+
20800|(262144,[571,1536...|
+-----+--------------------+
only showing top 1 row

In [18]:
train_data, test_data = data.randomSplit([0.75, 0.25], seed=12345)

In [19]:
from pyspark.ml.classification import LogisticRegression
model=LogisticRegression(labelCol='label',maxIter=5, regParam=0.001)           
model=model.fit(train_data)                                                          
summary=model.summary
summary.predictions.describe().show() 

+-------+-----------------+------------------+-------------------+
summary| id| label| prediction|
+-------+-----------------+------------------+-------------------+
 count| 15509| 15509| 15509|
 mean|10397.95318847121|0.4995164098265523|0.49925849506738024|
 stddev|5988.160077617101|0.5000158865849709| 0.5000155706044268|
 min| 0| 0.0| 0.0|
 max| 20799| 1.0| 1.0|
+-------+-----------------+------------------+-------------------+

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions=model.evaluate(test_data)

In [21]:
evaluator=BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
evaluator.evaluate(predictions.predictions)

Out[154]: 0.9630269710284715

In [22]:
#Achieved 96.3% accuracy on test data, lets predict on the unseen test data for kaggle submission.

In [23]:
results=model.transform(test_data)
results.select('features','prediction').show()

+--------------------+----------+
 features|prediction|
+--------------------+----------+
(262144,[358,1536...| 0.0|
(262144,[331,644,...| 0.0|
(262144,[1156,232...| 1.0|
(262144,[14,405,1...| 0.0|
(262144,[2711,408...| 0.0|
(262144,[3645,486...| 1.0|
(262144,[14,805,2...| 0.0|
(262144,[14,571,9...| 0.0|
(262144,[2970,357...| 0.0|
(262144,[192,736,...| 1.0|
(262144,[1846,243...| 0.0|
(262144,[9616,963...| 1.0|
(262144,[14,1304,...| 0.0|
(262144,[1001,191...| 0.0|
(262144,[511,632,...| 0.0|
(262144,[168,1226...| 1.0|
(262144,[4200,606...| 0.0|
(262144,[2851,417...| 0.0|
(262144,[14,632,1...| 0.0|
(262144,[4037,475...| 1.0|
+--------------------+----------+
only showing top 20 rows

In [24]:
results.count()

Out[157]: 5291

In [25]:
results=model.transform(testData)
results.select('features','prediction').show()

+--------------------+----------+
 features|prediction|
+--------------------+----------+
(262144,[571,1536...| 0.0|
(262144,[1536,565...| 1.0|
(262144,[1804,208...| 1.0|
(262144,[1536,222...| 0.0|
(262144,[528,4270...| 1.0|
(262144,[5795,761...| 1.0|
(262144,[15780,16...| 0.0|
(262144,[951,1001...| 1.0|
(262144,[364,571,...| 1.0|
(262144,[7658,761...| 1.0|
(262144,[92,693,1...| 1.0|
(262144,[1536,459...| 1.0|
(262144,[204,285,...| 1.0|
(262144,[1536,392...| 1.0|
(262144,[1156,153...| 1.0|
(262144,[1731,232...| 0.0|
(262144,[14,408,1...| 0.0|
(262144,[511,1536...| 0.0|
(262144,[1889,606...| 1.0|
(262144,[4927,784...| 0.0|
+--------------------+----------+
only showing top 20 rows

In [26]:
results.count()

Out[160]: 5200

In [27]:
values = list(results.select('prediction').toPandas()['prediction'])

In [28]:
print(values)

[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0,

In [29]:
import pandas as pd
res = pd.DataFrame(values)
res.head()

,0
0,0.0
1,1.0
2,1.0
3,0.0
4,1.0
